In [63]:
import pandas as pd

data = pd.read_csv('C:/Users/user/Desktop/Assignment/data.csv')
data.shape

(59827, 26)

In [67]:
data = data.sample(n=10000, random_state = 67)
#sample = data.sample(frac=0.8,random_state = 67)

In [65]:
data['driving'] = None
sample_list = sample.index
for i in sample_list:
    data['driving'][i] = 0

<ipython-input-65-d83bb74f4f19>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['driving'][i] = 0


In [66]:
#data = data.fillna(1)

In [71]:
#trip(운행횟수) 정의
data['trip'] = None

for i in data['carnum'].unique():
    data_num = data.loc[data['carnum']==i]
    data_num = data_num.sort_values(by='dprt_time')
    trip_num = 1
    for j in data_num.index:
        data['trip'][j] = trip_num
        trip_num +=1

<ipython-input-71-8db83f0cdddc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['trip'][j] = trip_num


In [74]:
import polyline
import folium
import requests
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    #steps = "?steps=true"
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc)#+steps) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

#df
data['routes'] = data.apply(lambda x: get_route(x['dprt_x'], 
                                            x['dprt_y'],
                                            x['arrvl_x'], 
                                            x['arrvl_y']), axis=1)


In [75]:
df = data.copy()

In [76]:
new_df = pd.DataFrame()
new_df['carnum'] = None
new_df['TIME'] = None
new_df['LAT'] = None
new_df['LNG'] = None
new_df['driving'] = None
new_df['trip'] = None

In [77]:
from datetime import timedelta
import time
from tqdm import tqdm
for i in tqdm(df.index):
    steps = "?steps=true"
    loc = "{},{};{},{}".format(df['dprt_x'][i], df['dprt_y'][i], df['arrvl_x'][i], df['arrvl_y'][i])
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc+steps) 
    res = r.json()  
    routes = polyline.decode(res['routes'][0]['geometry'])
    dprt_time =  pd.to_datetime(df['dprt_time'][i])
    start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
    end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
    end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
    car_num = df['carnum'][i]
    drive = df['driving'][i]
    trip = df['trip'][i]
    for j in range(len(routes)):
        lon = routes[j][1]
        lat = routes[j][0]
        time_dif = dprt_time + timedelta(seconds = end_division*j)
        new_df = new_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,drive,trip]], columns=['carnum','TIME','LNG','LAT','driving','trip']), ignore_index=True)
    

100%|██████████| 10000/10000 [10:12<00:00, 16.32it/s]


In [40]:
new_df['driving'].value_counts()

0    92688
1    23052
Name: driving, dtype: int64

In [46]:
new_df.shape

(232076, 6)

In [47]:
new_df

,carnum,TIME,LAT,LNG,driving,trip
0,대전50바1004,2017-03-02 00:40:04+00:00,36.32481,127.40056,0,NaN
1,대전50바1004,2017-03-02 00:40:36.166667+00:00,36.32704,127.39968,0,NaN
2,대전50바1004,2017-03-02 00:41:08.333333+00:00,36.32965,127.40847,0,NaN
3,대전50바1004,2017-03-02 00:41:40.500000+00:00,36.33458,127.41339,0,NaN
4,대전50바1004,2017-03-02 00:42:12.666667+00:00,36.34238,127.42331,0,NaN
...,...,...,...,...,...,...
232071,대전50바4034,2017-03-02 09:30:45.436364+00:00,36.35816,127.41069,0,4.0
232072,대전50바4034,2017-03-02 09:31:43.009091+00:00,36.35908,127.43219,0,4.0
232073,대전50바4034,2017-03-02 09:32:40.581818+00:00,36.35409,127.43838,0,4.0
232074,대전50바4034,2017-03-02 09:33:38.154545+00:00,36.35257,127.43658,0,4.0


In [54]:
new_df = new_df.loc[new_df['trip']>0]

In [78]:
from shapely.geometry import Point
from shapely.geometry import Polygon
new_df['geometry'] = new_df.apply(lambda x:Point(x['LNG'],x['LAT']),axis=1)
new_df['TIME'] = new_df['TIME'].apply(lambda x: pd.to_datetime(pd.to_datetime(x).strftime('%Y-%m-%d-%H:%M:%S')))


In [80]:
import pickle
## Save pickle
with open("C:/Users/user/Desktop/Assignment/new_df.pickle","wb") as fw:
    pickle.dump(new_df, fw)
 




In [2]:
random_passenger

,time,dprt_lat,dprt_lon,arrive_lat,arrive_lon,geometry
0,2017-03-02 06:26:00,36.293438,127.395359,36.359618,127.327551,"POLYGON ((127.4623063907231 36.29341965240555,..."
1,2017-03-02 06:26:00,36.368788,127.359499,36.365580,127.400152,"POLYGON ((127.4265113636116 36.36876902768055,..."
2,2017-03-02 06:26:00,36.356105,127.322142,36.403937,127.329913,"POLYGON ((127.3891433406189 36.35608601681965,..."
3,2017-03-02 06:11:00,36.325313,127.473995,36.319507,127.324998,"POLYGON ((127.5409693999005 36.32529453546276,..."
4,2017-03-02 06:11:00,36.292056,127.393915,36.372816,127.416656,"POLYGON ((127.4608607160801 36.29203707397843,..."
...,...,...,...,...,...,...
1615,2017-03-02 23:24:00,36.290431,127.426041,36.343654,127.448215,"POLYGON ((127.4929852528596 36.29041218255439,..."
1616,2017-03-02 23:24:00,36.300518,127.422913,36.309680,127.407640,"POLYGON ((127.4898663270284 36.30049957751347,..."
1617,2017-03-02 23:55:00,36.304289,127.351495,36.303448,127.399978,"POLYGON ((127.4184516285945 36.30427076316115,..."
1618,2017-03-02 23:55:00,36.339554,127.394463,36.327283,127.314273,"POLYGON ((127.46145041691 36.33953546030614, 1..."


In [1]:
import pickle
## Load pickle
with open("C:/Users/user/Desktop/Assignment/new_df.pickle","rb") as fr:
    new_df = pickle.load(fr)
    
import pickle
## Load pickle
with open("C:/Users/user/Desktop/Assignment/random_passenger.pickle","rb") as fr:
    random_passenger = pickle.load(fr)
 

def get_distance(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    steps = "?steps=true"
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc +steps) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    #duration = res['routes'][0]['duration']
    distance = int(round(res['routes'][0]['distance'],0))
    return distance

def save_solution(status):   
    i_list = []
    j_list = []
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        #print('Total distance = ', solver.Objective().Value(), '\n')
        for i in range(num_taxis):
            for j in range(num_passenger):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    #print('taxi %d assigned to passenger %d.  distance = %d' %
                        #(i, j, matrix_list[i][j]))   
                    i_list.append(i)
                    j_list.append(j)
    return i_list, j_list

import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
import geog
from tqdm import tqdm
random_passenger['geometry']=None
for i in tqdm(range(random_passenger.shape[0])):
    point = Point([random_passenger['dprt_lon'][i],random_passenger['dprt_lat'][i]])
    num_points = 36 # 점 개수
    radius = 6000 # 반경, 미터(meters)
    angles = np.linspace(0, 360, num_points)
    circle = Polygon(geog.propagate(point, angles, radius))
    random_passenger['geometry'][i] = circle

#random_passenger.to_csv('C:/Users/user/Desktop/Assignment/random_passenger.csv',encoding='utf-8')

  0%|          | 0/1620 [00:00<?, ?it/s]C:\Users\user\anaconda3\lib\site-packages\geog\geog.py:159: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  single, (p0, angle, d) = _to_arrays((p0, 2), (angle, 1), (d, 1))
<ipython-input-1-51e5bd795c2e>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_passenger['geometry'][i] = circle
100%|██████████| 1620/1620 [00:00<00:00, 2610.10it/s]


In [259]:
len(new_df['carnum'].unique())

2153

In [2]:
#같은 시간대에 여러개의 위치 생성
from random import *
from tqdm import tqdm
import pandas as pd
random_passenger = pd.DataFrame()
random_passenger['time'] = None
random_passenger['dprt_lat'] = None
random_passenger['dprt_lon'] = None
random_passenger['arrive_lat'] = None
random_passenger['arrive_lon'] = None
for hour in tqdm(range(6,24)):
    for i in range(30):
        minute = randint(0,59)
        time = pd.Timestamp(2017,3,2,hour,minute)
        for j in range(3):
            start_lat = uniform(36.28872150215423,36.411218105067874)
            start_lon = uniform(127.30846055394692,127.47428543061531)
            arrive_lat = uniform(36.28872150215423,36.411218105067874)
            arrive_lon = uniform(127.30846055394692,127.47428543061531)        
            random_passenger = random_passenger.append(pd.DataFrame([[time,start_lat,start_lon,arrive_lat,arrive_lon]],columns=['time','dprt_lat','dprt_lon','arrive_lat','arrive_lon']),ignore_index=True)
            #new_df = new_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,drive]], columns=['carnum','TIME','LNG','LAT','driving']), ignore_index=True)

100%|██████████| 18/18 [00:01<00:00, 16.80it/s]


In [3]:
import pickle
## Save pickle
with open("C:/Users/user/Desktop/Assignment/random_passenger.pickle","wb") as fw:
    pickle.dump(random_passenger, fw)
 




In [3]:
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
import geog
random_passenger['geometry']=None

for i in tqdm(range(random_passenger.shape[0])):
    point = Point([random_passenger['dprt_lon'][i],random_passenger['dprt_lat'][i]])
    num_points = 36 # 점 개수
    radius = 1000 # 반경, 미터(meters)
    angles = np.linspace(0, 360, num_points)
    circle = Polygon(geog.propagate(point, angles, radius))
    random_passenger['geometry'][i] = circle

random_passenger.to_csv('C:/Users/user/Desktop/Assignment/random_passenger.csv',encoding='utf-8')

  0%|          | 0/1620 [00:00<?, ?it/s]C:\Users\user\anaconda3\lib\site-packages\geog\geog.py:159: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  single, (p0, angle, d) = _to_arrays((p0, 2), (angle, 1), (d, 1))
<ipython-input-3-870e9ceb03c1>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_passenger['geometry'][i] = circle
100%|██████████| 1620/1620 [00:00<00:00, 2665.80it/s]


In [3]:
def get_distance(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    steps = "?steps=true"
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc +steps) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    #duration = res['routes'][0]['duration']
    distance = int(round(res['routes'][0]['distance'],0))
    return distance

In [4]:
def save_solution(status):   
    i_list = []
    j_list = []
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        #print('Total distance = ', solver.Objective().Value(), '\n')
        for i in range(num_taxis):
            for j in range(num_passenger):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    #print('taxi %d assigned to passenger %d.  distance = %d' %
                    #    (i, j, matrix_list[i][j]))   
                    i_list.append(i)
                    j_list.append(j)
    return i_list, j_list

In [14]:
print(i)

49


In [ ]:
#1차 코드(연산량이 너무 많음, driving=2인 경우에 driving =1에 소요되는 시간이 더해지지 않음)
import requests
from tqdm import tqdm
from datetime import timedelta
import polyline
import folium
import time
from ortools.linear_solver import pywraplp
result_df = pd.DataFrame()
result_df['carnum'] = None
result_df['TIME'] = None
result_df['LAT'] = None
result_df['LNG'] = None
result_df['trip'] = None
result_df['driving'] = None
for i in tqdm(random_passenger['time'].unique()):
    #time.sleep(90)
    matrix_list = []
    time_i = pd.Timestamp(i)
    time_df = new_df.loc[(new_df['TIME']> time_i - timedelta(minutes=1))&(new_df['TIME']< time_i + timedelta(minutes=1))]
    random_ = random_passenger.loc[random_passenger['time']==time_i]

    for j in time_df.index:
        distance_list = []
        for k in random_.index:
            a = get_distance(time_df['LNG'][j],time_df['LAT'][j],random_['dprt_lon'][k],random_['dprt_lat'][k])
            distance_list.append(a)
        matrix_list.append(distance_list)
    
    

    num_taxis = len(matrix_list)
    num_passenger = len(matrix_list[0])

    #Declare the MIP solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    #Create the variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_taxis):
        for j in range(num_passenger):
            x[i, j] = solver.IntVar(0, 1, '')

    #Create the constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_taxis):
        solver.Add(solver.Sum([x[i, j] for j in range(num_passenger)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_passenger):
        solver.Add(solver.Sum([x[i, j] for i in range(num_taxis)]) == 1)

    #Create the objective function
    objective_terms = []
    for i in range(num_taxis):
        for j in range(num_passenger):
            objective_terms.append(matrix_list[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))
    #목적함수의 값은 solver에 의해 값 1이 할당된 모든 변수의 총 비용

    #Invoke the solver
    status = solver.Solve()

    a,b = save_solution(status)
    for i in range(len(a)):
        steps = "?steps=true"
        loc = "{},{};{},{}".format(time_df['LNG'][time_df.index[a[i]]], 
                                    time_df['LAT'][time_df.index[a[i]]], 
                                    random_['dprt_lon'][random_.index[b[i]]], random_['dprt_lat'][random_.index[b[i]]])
        url = "http://127.0.0.1:5000/route/v1/driving/"
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        dprt_time =  pd.to_datetime(time_df['TIME'][time_df.index[a[i]]])
        start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
        end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
        car_num = time_df['carnum'][time_df.index[a[i]]]
        drive = 1
        for j in range(len(routes)):
            lon = routes[j][1]
            lat = routes[j][0]
            time_dif = dprt_time + timedelta(seconds = end_division*j)
            result_df = result_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,time_df['trip'][time_df.index[a[i]]],drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        #        print(i)


        loc = "{},{};{},{}".format( random_['dprt_lon'][random_.index[b[i]]], random_['dprt_lat'][random_.index[b[i]]],
                                    random_['arrive_lon'][random_.index[b[i]]],random_['arrive_lat'][random_.index[b[i]]])
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        dprt_time =  pd.to_datetime(random_['time'][random_.index[b[i]]])
        start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
        end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
        car_num = time_df['carnum'][time_df.index[a[i]]]
        drive = 2
        for j in range(len(routes)):
            lon = routes[j][1]
            lat = routes[j][0]
            time_dif = dprt_time + timedelta(seconds = end_division*j)
            result_df = result_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,time_df['trip'][time_df.index[a[i]]],drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        





In [ ]:
#2차 코드(연산량이 너무 많음)
import requests
from tqdm import tqdm
from datetime import timedelta
import polyline
import folium
import time
from ortools.linear_solver import pywraplp
result_df = pd.DataFrame()
result_df['carnum'] = None
result_df['TIME'] = None
result_df['LAT'] = None
result_df['LNG'] = None
result_df['trip'] = None
result_df['driving'] = None
for i in tqdm(random_passenger['time'].unique()):
    #time.sleep(90)
    matrix_list = []
    time_i = pd.Timestamp(i)
    time_df = new_df.loc[(new_df['TIME']> time_i - timedelta(minutes=1))&(new_df['TIME']< time_i + timedelta(minutes=1))]
    random_ = random_passenger.loc[random_passenger['time']==time_i]

    for j in time_df.index:
        distance_list = []
        for k in random_.index:
            a = get_distance(time_df['LNG'][j],time_df['LAT'][j],random_['dprt_lon'][k],random_['dprt_lat'][k])
            distance_list.append(a)
        matrix_list.append(distance_list)
    
    

    num_taxis = len(matrix_list)
    num_passenger = len(matrix_list[0])

    #Declare the MIP solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    #Create the variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_taxis):
        for j in range(num_passenger):
            x[i, j] = solver.IntVar(0, 1, '')

    #Create the constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_taxis):
        solver.Add(solver.Sum([x[i, j] for j in range(num_passenger)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_passenger):
        solver.Add(solver.Sum([x[i, j] for i in range(num_taxis)]) == 1)

    #Create the objective function
    objective_terms = []
    for i in range(num_taxis):
        for j in range(num_passenger):
            objective_terms.append(matrix_list[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))
    #목적함수의 값은 solver에 의해 값 1이 할당된 모든 변수의 총 비용

    #Invoke the solver
    status = solver.Solve()

    a,b = save_solution(status)
    for i in range(len(a)):
        steps = "?steps=true"
        loc = "{},{};{},{}".format(time_df['LNG'][time_df.index[a[i]]], 
                                    time_df['LAT'][time_df.index[a[i]]], 
                                    random_['dprt_lon'][random_.index[b[i]]], random_['dprt_lat'][random_.index[b[i]]])
        url = "http://127.0.0.1:5000/route/v1/driving/"
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        pickup_time =  pd.to_datetime(time_df['TIME'][time_df.index[a[i]]])
        pickup_start = pickup_time.strftime('%Y-%m-%d-%H:%M:%S')
        duration = res['routes'][0]['legs'][0]['duration']
        pickup_end = pd.to_datetime(pickup_time + timedelta(seconds=duration)).strftime('%Y-%m-%d-%H:%M:%S')
        pickup_end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
        car_num = time_df['carnum'][time_df.index[a[i]]]
        drive = 1
        for j in range(len(routes)):
            lon = routes[j][1]
            lat = routes[j][0]
            time_dif = pickup_time + timedelta(seconds = pickup_end_division*j)
            result_df = result_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,time_df['trip'][time_df.index[a[i]]],drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        #        print(i)


        loc = "{},{};{},{}".format( random_['dprt_lon'][random_.index[b[i]]], random_['dprt_lat'][random_.index[b[i]]],
                                    random_['arrive_lon'][random_.index[b[i]]],random_['arrive_lat'][random_.index[b[i]]])
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        dprt_time =  pd.to_datetime(random_['time'][random_.index[b[i]]]+timedelta(seconds=duration))
        #start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
        end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
        car_num = time_df['carnum'][time_df.index[a[i]]]
        drive = 2
        for j in range(len(routes)):
            lon = routes[j][1]
            lat = routes[j][0]
            time_dif = dprt_time + timedelta(seconds = end_division*j)
            result_df = result_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,time_df['trip'][time_df.index[a[i]]],drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        result_df.to_csv('C:/Users/user/Desktop/Assignment/result_df22.csv',encoding='utf-8')





In [7]:
len(result_df['carnum'].unique())

113

In [5]:
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
import geog

random_passenger['geometry']=None
for i in tqdm(range(random_passenger.shape[0])):
    point = Point([random_passenger['dprt_lon'][i],random_passenger['dprt_lat'][i]])
    num_points = 36 # 점 개수
    radius = 5000 # 반경, 미터(meters)
    angles = np.linspace(0, 360, num_points)
    circle = Polygon(geog.propagate(point, angles, radius))
    random_passenger['geometry'][i] = circle

random_passenger.to_csv('C:/Users/user/Desktop/Assignment/random_passenger.csv',encoding='utf-8')

  0%|          | 0/1620 [00:00<?, ?it/s]C:\Users\user\anaconda3\lib\site-packages\geog\geog.py:159: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  single, (p0, angle, d) = _to_arrays((p0, 2), (angle, 1), (d, 1))
<ipython-input-5-5c0453e9e08c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_passenger['geometry'][i] = circle
100%|██████████| 1620/1620 [00:00<00:00, 2657.24it/s]


In [244]:
#3차
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
from ortools.linear_solver import pywraplp
import geog
import geopandas as gpd
result_df = pd.DataFrame()
result_df['carnum'] = None
result_df['TIME'] = None
result_df['LAT'] = None
result_df['LNG'] = None
result_df['trip'] = None
result_df['driving'] = None
random_passenger['geometry']=None


for i in tqdm(random_passenger['time'].unique()):
    matrix_list = []
    time_i = pd.Timestamp(i)
    time_df = new_df.loc[(new_df['TIME']> time_i - timedelta(minutes=1))&(new_df['TIME']< time_i + timedelta(minutes=1))]
    random_ = random_passenger.loc[random_passenger['time']==time_i]
    time_df_unique = time_df['carnum'].unique()
    for j in range(random_.shape[0]):
        geo_rand = gpd.GeoDataFrame(random_[j:j+1])
        geo_df = gpd.GeoDataFrame(time_df)
        df_join = gpd.sjoin(geo_rand,geo_df,how='left',predicate='intersects')
        join_df_unique = df_join['carnum'].unique()
        matrix_list = []
        for i in time_df.index:
            distance_list = []
            for j in random_.index:
                if time_df['carnum'][i] in list(df_join['carnum']):
                    distance_ = get_distance(time_df['LNG'][i],time_df['LAT'][i],
                                                random_['dprt_lon'][j],random_['dprt_lat'][j])
                    distance_list.append(distance_)
                else :
                    distance_list.append(99999)
            #print(distance_list)
            matrix_list.append(distance_list)

            

    num_taxis = len(matrix_list)
    num_passenger = len(matrix_list[0])

    #Declare the MIP solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    #Create the variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_taxis):
        for j in range(num_passenger):
            x[i, j] = solver.IntVar(0, 1, '')

    #Create the constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_taxis):
        solver.Add(solver.Sum([x[i, j] for j in range(num_passenger)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_passenger):
        solver.Add(solver.Sum([x[i, j] for i in range(num_taxis)]) == 1)

    #Create the objective function
    objective_terms = []
    for i in range(num_taxis):
        for j in range(num_passenger):
            objective_terms.append(matrix_list[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))
    #목적함수의 값은 solver에 의해 값 1이 할당된 모든 변수의 총 비용

    #Invoke the solver
    status = solver.Solve()

    a,b = save_solution(status)  
    
    for i in range(len(a)):
        steps = "?steps=true"
        loc = "{},{};{},{}".format(time_df['LNG'][time_df.index[a[i]]], 
                                    time_df['LAT'][time_df.index[a[i]]], 
                                    random_['dprt_lon'][random_.index[b[i]]], random_['dprt_lat'][random_.index[b[i]]])
        url = "http://127.0.0.1:5000/route/v1/driving/"
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        pickup_time =  pd.to_datetime(time_df['TIME'][time_df.index[a[i]]])
        pickup_start = pickup_time.strftime('%Y-%m-%d-%H:%M:%S')
        duration = res['routes'][0]['legs'][0]['duration']
        pickup_end = pd.to_datetime(pickup_time + timedelta(seconds=duration)).strftime('%Y-%m-%d-%H:%M:%S')
        pickup_end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
        car_num = time_df['carnum'][time_df.index[a[i]]]
        drive = 1
        for j in range(len(routes)):
            lon = routes[j][1]
            lat = routes[j][0]
            time_dif = pickup_time + timedelta(seconds = pickup_end_division*j)
            result_df = result_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,time_df['trip'][time_df.index[a[i]]],drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        #        print(i)


        loc = "{},{};{},{}".format( random_['dprt_lon'][random_.index[b[i]]], random_['dprt_lat'][random_.index[b[i]]],
                                    random_['arrive_lon'][random_.index[b[i]]],random_['arrive_lat'][random_.index[b[i]]])
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        dprt_time =  pd.to_datetime(random_['time'][random_.index[b[i]]]+timedelta(seconds=duration))
        #start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
        end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        end_division = res['routes'][0]['legs'][0]['duration']/len(routes)
        car_num = time_df['carnum'][time_df.index[a[i]]]
        drive = 2
        for j in range(len(routes)):
            lon = routes[j][1]
            lat = routes[j][0]
            time_dif = dprt_time + timedelta(seconds = end_division*j)
            result_df = result_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,time_df['trip'][time_df.index[a[i]]],drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
    result_df.to_csv('C:/Users/user/Desktop/Assignment/result_df.csv',encoding='utf-8')


  
            

100%|██████████| 438/438 [05:04<00:00,  1.44it/s]


In [205]:
result_df['carnum'].value_counts()

대전50바2921    54
대전50바3456    50
대전50바1936    42
대전50바6304    37
대전50바4426    30
대전50바3244    28
Name: carnum, dtype: int64

In [248]:
result_df.to_csv('C:/Users/user/Desktop/Assignment/result_df.csv',encoding='utf-8')
#a.to_csv('C:/Users/user/Desktop/Assignment/a.csv',encoding='utf-8')

In [221]:
result_df.shape[0]

66435

In [220]:
len(result_df['carnum'].unique())

283

In [8]:
new_df = pd.DataFrame()
new_df['carnum'] = None
new_df['pickup_time'] = None
new_df['dropoff_time'] = None
new_df['pickup_lon'] = None
new_df['pickup_lat'] = None
new_df['dropoff_lon'] = None
new_df['dropoff_lat'] = None

In [224]:
result_df.loc[result_df['carnum']==result_df['carnum'].unique()[0]]

,carnum,TIME,LAT,LNG,trip,driving
0,대전50바5123,2017-03-02 06:19:21.000000,36.35647,127.43284,1.0,1
1,대전50바5123,2017-03-02 06:20:10.215385,36.37459,127.42795,1.0,1
2,대전50바5123,2017-03-02 06:20:59.430769,36.37312,127.42432,1.0,1
3,대전50바5123,2017-03-02 06:21:48.646154,36.37326,127.42017,1.0,1
4,대전50바5123,2017-03-02 06:22:37.861538,36.37461,127.41850,1.0,1
...,...,...,...,...,...,...
3343,대전50바5123,2017-03-02 06:36:05.088235,36.34114,127.31983,1.0,2
3344,대전50바5123,2017-03-02 06:37:01.970588,36.32992,127.31307,1.0,2
3345,대전50바5123,2017-03-02 06:37:58.852941,36.32533,127.31222,1.0,2
3346,대전50바5123,2017-03-02 06:38:55.735294,36.31989,127.31381,1.0,2


,carnum,TIME,LAT,LNG,trip,driving


In [9]:
for carnum in tqdm(result_df['carnum'].unique()):
    car_df = result_df.loc[result_df['carnum']==carnum].reset_index()
    for carnum_i in range(1,car_df.shape[0]):
        pickup_time = result_df['TIME'][carnum_i-1]
        dropoff_time = result_df['TIME'][carnum_i]
        pickup_lon = result_df['LNG'][carnum_i-1]
        pickup_lat = result_df['LAT'][carnum_i-1]
        dropoff_lon = result_df['LNG'][carnum_i]
        dropoff_lat = result_df['LAT'][carnum_i]
        new_df = new_df.append(pd.DataFrame([[carnum,pickup_time, dropoff_time,
                                                pickup_lon,pickup_lat,
                                                dropoff_lon,dropoff_lat]], 
                                            columns=['carnum','pickup_time', 
                                                        'dropoff_time','pickup_lon',
                                                        'pickup_lat','dropoff_lon',
                                                        'dropoff_lat']), ignore_index=True)

100%|██████████| 113/113 [00:06<00:00, 18.59it/s]


In [250]:
for carnum in tqdm(result_df['carnum'].unique()):
    car_df = result_df.loc[result_df['carnum']==carnum].reset_index()
    for carnum_i in range(car_df.shape[0]):
        if carnum_i == 0 :
            pickup_time = result_df['TIME'][carnum_i]
            dropoff_time = result_df['TIME'][carnum_i]
            pickup_lon = result_df['LNG'][carnum_i]
            pickup_lat = result_df['LAT'][carnum_i]
            dropoff_lon = result_df['LNG'][carnum_i]
            dropoff_lat = result_df['LAT'][carnum_i]
            new_df = new_df.append(pd.DataFrame([[carnum,pickup_time, dropoff_time,
                                                  pickup_lon,pickup_lat,
                                                  dropoff_lon,dropoff_lat]], 
                                                columns=['carnum','pickup_time', 
                                                         'dropoff_time','pickup_lon',
                                                         'pickup_lat','dropoff_lon',
                                                         'dropoff_lat']), ignore_index=True)
        else :
            pickup_time = result_df['TIME'][carnum_i-1]
            dropoff_time = result_df['TIME'][carnum_i]
            pickup_lon = result_df['LNG'][carnum_i-1]
            pickup_lat = result_df['LAT'][carnum_i-1]
            dropoff_lon = result_df['LNG'][carnum_i]
            dropoff_lat = result_df['LAT'][carnum_i]
            new_df = new_df.append(pd.DataFrame([[carnum,pickup_time, dropoff_time,
                                                  pickup_lon,pickup_lat,
                                                  dropoff_lon,dropoff_lat]], 
                                                columns=['carnum','pickup_time', 
                                                         'dropoff_time','pickup_lon',
                                                         'pickup_lat','dropoff_lon',
                                                         'dropoff_lat']), ignore_index=True)

100%|██████████| 283/283 [01:22<00:00,  3.42it/s]


# 22.02.04

In [18]:
import pickle
## Load pickle
with open("C:/Users/user/Desktop/Assignment/data.pickle","rb") as fr:
    new_df = pickle.load(fr)

In [20]:
new_df

,carnum,TIME,LAT,LNG,driving,trip,geometry
115740,대전50바6628,2017-03-02 17:26:39,36.32136,127.40899,1,1.0,POINT (127.40899 36.32136)
115741,대전50바6628,2017-03-02 17:26:59,36.32246,127.41259,1,1.0,POINT (127.41259 36.32246)
115742,대전50바6628,2017-03-02 17:27:20,36.32825,127.40551,1,1.0,POINT (127.40551 36.32825)
115743,대전50바6628,2017-03-02 17:27:41,36.32854,127.40551,1,1.0,POINT (127.40551 36.32854)
115744,대전50바6628,2017-03-02 17:28:02,36.32876,127.40562,1,1.0,POINT (127.40562 36.32876)
...,...,...,...,...,...,...,...
232071,대전50바4034,2017-03-02 09:30:45,36.35816,127.41069,0,4.0,POINT (127.41069 36.35816)
232072,대전50바4034,2017-03-02 09:31:43,36.35908,127.43219,0,4.0,POINT (127.43219 36.35908)
232073,대전50바4034,2017-03-02 09:32:40,36.35409,127.43838,0,4.0,POINT (127.43838 36.35409)
232074,대전50바4034,2017-03-02 09:33:38,36.35257,127.43658,0,4.0,POINT (127.43658 36.35257)


In [24]:
reshape_newdf = pd.DataFrame()
reshape_newdf['carnum'] = None
reshape_newdf['pickup_time'] = None
reshape_newdf['dropoff_time'] = None
reshape_newdf['pickup_lon'] = None
reshape_newdf['pickup_lat'] = None
reshape_newdf['dropoff_lon'] = None
reshape_newdf['dropoff_lat'] = None

In [22]:
new_df

,carnum,TIME,LAT,LNG,driving,trip,geometry
115740,대전50바6628,2017-03-02 17:26:39,36.32136,127.40899,1,1.0,POINT (127.40899 36.32136)
115741,대전50바6628,2017-03-02 17:26:59,36.32246,127.41259,1,1.0,POINT (127.41259 36.32246)
115742,대전50바6628,2017-03-02 17:27:20,36.32825,127.40551,1,1.0,POINT (127.40551 36.32825)
115743,대전50바6628,2017-03-02 17:27:41,36.32854,127.40551,1,1.0,POINT (127.40551 36.32854)
115744,대전50바6628,2017-03-02 17:28:02,36.32876,127.40562,1,1.0,POINT (127.40562 36.32876)
...,...,...,...,...,...,...,...
232071,대전50바4034,2017-03-02 09:30:45,36.35816,127.41069,0,4.0,POINT (127.41069 36.35816)
232072,대전50바4034,2017-03-02 09:31:43,36.35908,127.43219,0,4.0,POINT (127.43219 36.35908)
232073,대전50바4034,2017-03-02 09:32:40,36.35409,127.43838,0,4.0,POINT (127.43838 36.35409)
232074,대전50바4034,2017-03-02 09:33:38,36.35257,127.43658,0,4.0,POINT (127.43658 36.35257)


In [31]:
new_df = new_df.reset_index(drop=True)
for carnum in tqdm(new_df['carnum'].unique()):
    car_df = new_df.loc[new_df['carnum']==carnum].reset_index(drop=True)
    for carnum_i in range(1,car_df.shape[0]):
        pickup_time = new_df['TIME'][carnum_i-1]
        dropoff_time = new_df['TIME'][carnum_i]
        pickup_lon = new_df['LNG'][carnum_i-1]
        pickup_lat = new_df['LAT'][carnum_i-1]
        dropoff_lon = new_df['LNG'][carnum_i]
        dropoff_lat = new_df['LAT'][carnum_i]
        reshape_newdf = reshape_newdf.append(pd.DataFrame([[carnum,pickup_time, dropoff_time,
                                                pickup_lon,pickup_lat,
                                                dropoff_lon,dropoff_lat]], 
                                            columns=['carnum','pickup_time', 
                                                        'dropoff_time','pickup_lon',
                                                        'pickup_lat','dropoff_lon',
                                                        'dropoff_lat']), ignore_index=True)

100%|██████████| 2153/2153 [03:25<00:00, 10.49it/s]


In [32]:
reshape_newdf.to_csv('C:/Users/user/Desktop/Assignment/reshape_newdf.csv',encoding='utf-8')

In [94]:
import pandas as pd
reshape_newdf = pd.read_csv('C:/Users/user/Desktop/Assignment/reshape_newdf.csv',encoding='utf-8')

In [2]:
new_df['drving'] = 1

In [2]:
#4차 triplayer
import time
from tqdm import tqdm
import json
features = []
from datetime import timedelta
import geopandas as gpd
import requests
from ortools.linear_solver import pywraplp
import polyline
import folium
import pandas as pd
import requests



for i in tqdm(random_passenger['time'].unique()[:3]):
    time.sleep(30)
    time_i = pd.Timestamp(i)
    time_df = new_df.loc[(new_df['TIME']> time_i - timedelta(minutes=1))&(new_df['TIME']< time_i + timedelta(minutes=1))]
    random_ = random_passenger.loc[random_passenger['time']==time_i]
    matrix_list= []
    for i in time_df.index:
        distance_list=[]
        for random_index in random_.index:
            num = 0
            geo_rand = gpd.GeoDataFrame(random_[num:num+1])
            geo_df = gpd.GeoDataFrame(time_df)
            df_join = gpd.sjoin(geo_rand,geo_df,how='left',predicate='intersects')
            #print(df_join)
            num += 1
            if time_df['carnum'][i] in list(df_join['carnum'].unique()):
                distance_ = get_distance(time_df['LNG'][i],time_df['LAT'][i],
                                            random_['dprt_lon'][random_index],
                                            random_['dprt_lat'][random_index])
                distance_list.append(distance_)
            else :
                distance_list.append(99999)
        matrix_list.append(distance_list)

            

    num_taxis = len(matrix_list)
    num_passenger = len(matrix_list[0])

    #Declare the MIP solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    #Create the variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_taxis):
        for j in range(num_passenger):
            x[i, j] = solver.IntVar(0, 1, '')

    #Create the constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_taxis):
        solver.Add(solver.Sum([x[i, j] for j in range(num_passenger)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_passenger):
        solver.Add(solver.Sum([x[i, j] for i in range(num_taxis)]) == 1)

    #Create the objective function
    objective_terms = []
    for i in range(num_taxis):
        for j in range(num_passenger):
            objective_terms.append(matrix_list[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))
    #목적함수의 값은 solver에 의해 값 1이 할당된 모든 변수의 총 비용

    #Invoke the solver
    status = solver.Solve()

    a,b = save_solution(status)  
    
    for i in a:
        passenger_ = a.index(i)
        pickup_time = pd.to_datetime(time_df['TIME'][time_df.index[i]]+timedelta(hours=9))
        car_num = time_df['carnum'][time_df.index[i]]
        trip_num = time_df['trip'][time_df.index[i]]
        new_df_index = new_df.loc[(new_df['TIME']>pickup_time)&(new_df['carnum']==car_num)&(new_df['trip']==trip_num)].index.tolist()
        new_df = new_df.loc[~new_df.index.isin(new_df_index)]
        steps = "?steps=true"
        loc = "{},{};{},{}".format(time_df['LNG'][time_df.index[i]], 
                                    time_df['LAT'][time_df.index[i]], 
                                    random_['dprt_lon'][random_.index[passenger_]], random_['dprt_lat'][random_.index[passenger_]])
        url = "http://127.0.0.1:5000/route/v1/driving/"
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        start_time = pickup_time.strftime('%Y-%m-%d-%H:%M:%S')
        #duration_time =  pd.to_datetime(df['dprt_time'][i])
        end_time = pd.to_datetime(pickup_time + timedelta(hours=9,seconds=res['routes'][0]['legs'][0]['duration']))
        #end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        route_gpd = gpd.GeoDataFrame(res['routes'][0]['legs'][0]['steps'])
        #duration_time =  pd.to_datetime(time_df['TIME'][time_df.index[a[i]]]) + timedelta(hours=9)
        drive = 2
        for j in range(route_gpd.shape[0]-1):
            detail_route = polyline.decode(route_gpd['geometry'][j])
            detail_duration = route_gpd['duration'][j] /len(detail_route)
            coordinates_list = []
            for k in range(len(detail_route)):
                lon = detail_route[k][1]
                lat = detail_route[k][0]
                duration_time = pickup_time + timedelta(seconds = detail_duration)
                pickup_time = duration_time
                alt = 0
                time_diff = int(time.mktime(duration_time.timetuple()))
                coordinates = [lon,lat,alt,time_diff]
                coordinates_list.append(coordinates)
            features.append(
            {
                "type": "Feature",
                "status" : drive,
                "properties": {
                "start_time": start_time,
                #"end_time": end_time,
                "trip_uuid": car_num
                },
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates_list
                }
            }
        )
    
        ###############################################
        loc = "{},{};{},{}".format( random_['dprt_lon'][random_.index[passenger_]], random_['dprt_lat'][random_.index[passenger_]],
                                    random_['arrive_lon'][random_.index[passenger_]],random_['arrive_lat'][random_.index[passenger_]])
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        dprt_time =  end_time
        start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
        #duration_time =  pd.to_datetime(df['dprt_time'][i])
        #end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        route_gpd = gpd.GeoDataFrame(res['routes'][0]['legs'][0]['steps'])
        #duration_time =  pd.to_datetime(df['dprt_time'][i]) + timedelta(hours=9)
        drive = 1
        for j in range(route_gpd.shape[0]-1):
            detail_route = polyline.decode(route_gpd['geometry'][j])
            detail_duration = route_gpd['duration'][j] /len(detail_route)
            for k in range(len(detail_route)):
                lon = detail_route[k][1]
                lat = detail_route[k][0]
                duration_time = dprt_time + timedelta(seconds = detail_duration)
                dprt_time = duration_time
                time_diff = int(time.mktime(duration_time.timetuple()))
                coordinates = [lon,lat,alt,time_diff]
                coordinates_list.append(coordinates)
            features.append(
            {
                "type": "Feature",
                "status" : drive,
                "properties": {
                "start_time": start_time,
                #"end_time": end_time,
                "trip_uuid": car_num
                },
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates_list
                }
            }
            )    
        with open('C:/Users/user/Desktop/Assignment/abc.json', 'w') as outfile:
            json.dump(features, outfile, indent=4)


100%|██████████| 3/3 [01:37<00:00, 32.50s/it]


In [259]:
new_df['driving'] = 1

In [5]:
import pickle
## Load pickle
with open("C:/Users/user/Desktop/Assignment/new_df.pickle","rb") as fr:
    new_df = pickle.load(fr)
    
import pickle
## Load pickle
with open("C:/Users/user/Desktop/Assignment/random_passenger.pickle","rb") as fr:
    random_passenger = pickle.load(fr)
 

def get_distance(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    steps = "?steps=true"
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc +steps) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    #duration = res['routes'][0]['duration']
    distance = int(round(res['routes'][0]['distance'],0))
    return distance

def save_solution(status):   
    i_list = []
    j_list = []
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        #print('Total distance = ', solver.Objective().Value(), '\n')
        for i in range(num_taxis):
            for j in range(num_passenger):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    #print('taxi %d assigned to passenger %d.  distance = %d' %
                        #(i, j, matrix_list[i][j]))   
                    i_list.append(i)
                    j_list.append(j)
    return i_list, j_list

import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
import geog
from tqdm import tqdm
random_passenger['geometry']=None
for i in tqdm(range(random_passenger.shape[0])):
    point = Point([random_passenger['dprt_lon'][i],random_passenger['dprt_lat'][i]])
    num_points = 36 # 점 개수
    radius = 6000 # 반경, 미터(meters)
    angles = np.linspace(0, 360, num_points)
    circle = Polygon(geog.propagate(point, angles, radius))
    random_passenger['geometry'][i] = circle

#random_passenger.to_csv('C:/Users/user/Desktop/Assignment/random_passenger.csv',encoding='utf-8')

  0%|          | 0/1620 [00:00<?, ?it/s]C:\Users\user\anaconda3\lib\site-packages\geog\geog.py:159: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  single, (p0, angle, d) = _to_arrays((p0, 2), (angle, 1), (d, 1))
<ipython-input-5-51e5bd795c2e>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_passenger['geometry'][i] = circle
100%|██████████| 1620/1620 [00:00<00:00, 2621.31it/s]


In [ ]:
#5차
import numpy as np
from datetime import timedelta
from shapely.geometry import Point
from shapely.geometry import Polygon
from ortools.linear_solver import pywraplp
import geog
import geopandas as gpd
import polyline
import folium
import requests
import pandas as pd
from tqdm import tqdm

for i in tqdm(random_passenger['time'].unique()):
    time.sleep(15)
    time_i = pd.Timestamp(i)
    time_df = new_df.loc[(new_df['TIME']> time_i - timedelta(minutes=1))&(new_df['TIME']< time_i + timedelta(minutes=1))]
    random_ = random_passenger.loc[random_passenger['time']==time_i]
    matrix_list= []
    for i in time_df.index:
        distance_list=[]
        for random_index in random_.index:
            num = 0
            geo_rand = gpd.GeoDataFrame(random_[num:num+1])
            geo_df = gpd.GeoDataFrame(time_df)
            df_join = gpd.sjoin(geo_rand,geo_df,how='left',predicate='intersects')
            #print(df_join)
            num += 1
            if time_df['carnum'][i] in list(df_join['carnum'].unique()):
                distance_ = get_distance(time_df['LNG'][i],time_df['LAT'][i],
                                            random_['dprt_lon'][random_index],
                                            random_['dprt_lat'][random_index])
                distance_list.append(distance_)
            else :
                distance_list.append(99999)
        matrix_list.append(distance_list)

            

    num_taxis = len(matrix_list)
    num_passenger = len(matrix_list[0])

    #Declare the MIP solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    #Create the variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_taxis):
        for j in range(num_passenger):
            x[i, j] = solver.IntVar(0, 1, '')

    #Create the constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_taxis):
        solver.Add(solver.Sum([x[i, j] for j in range(num_passenger)]) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_passenger):
        solver.Add(solver.Sum([x[i, j] for i in range(num_taxis)]) == 1)

    #Create the objective function
    objective_terms = []
    for i in range(num_taxis):
        for j in range(num_passenger):
            objective_terms.append(matrix_list[i][j] * x[i, j])
    solver.Minimize(solver.Sum(objective_terms))
    #목적함수의 값은 solver에 의해 값 1이 할당된 모든 변수의 총 비용

    #Invoke the solver
    status = solver.Solve()

    a,b = save_solution(status)  
    for i in a:
        passenger_ = a.index(i)
        pickup_time = pd.to_datetime(time_df['TIME'][time_df.index[i]])
        car_num = time_df['carnum'][time_df.index[i]]
        trip_num = time_df['trip'][time_df.index[i]]
        new_df_index = new_df.loc[(new_df['TIME']>pickup_time)&(new_df['carnum']==car_num)&(new_df['trip']==trip_num)].index.tolist()
        new_df = new_df.loc[~new_df.index.isin(new_df_index)]
        steps = "?steps=true"
        loc = "{},{};{},{}".format(time_df['LNG'][time_df.index[i]], 
                                    time_df['LAT'][time_df.index[i]], 
                                    random_['dprt_lon'][random_.index[passenger_]], random_['dprt_lat'][random_.index[passenger_]])
        url = "http://127.0.0.1:5000/route/v1/driving/"
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        start_time = pickup_time.strftime('%Y-%m-%d-%H:%M:%S')
        #duration_time =  pd.to_datetime(df['dprt_time'][i])
        end_time = pd.to_datetime(pickup_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration']))
        #end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        route_gpd = gpd.GeoDataFrame(res['routes'][0]['legs'][0]['steps'])
        #duration_time =  pd.to_datetime(time_df['TIME'][time_df.index[a[i]]]) + timedelta(hours=9)
        drive = 2
        passenger = '탑승객'
        for j in range(route_gpd.shape[0]-1):
            detail_route = polyline.decode(route_gpd['geometry'][j])
            detail_duration = route_gpd['duration'][j] /len(detail_route)
            for k in range(len(detail_route)):
                lon = detail_route[k][1]
                lat = detail_route[k][0]
                duration_time = pickup_time + timedelta(seconds = detail_duration)
                pickup_time = duration_time
                new_df = new_df.append(pd.DataFrame([[car_num,duration_time,lon,lat,time_df['trip'][time_df.index[i]]+1,drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
                new_df=new_df.append(pd.DataFrame([[passenger,duration_time,random_['dprt_lon'][random_.index[passenger_]],random_['dprt_lat'][random_.index[passenger_]],time_df['trip'][time_df.index[i]]+1,3]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        loc = "{},{};{},{}".format( random_['dprt_lon'][random_.index[passenger_]], random_['dprt_lat'][random_.index[passenger_]],
                                    random_['arrive_lon'][random_.index[passenger_]],random_['arrive_lat'][random_.index[passenger_]])
        r = requests.get(url + loc+steps) 
        res = r.json()  
        routes = polyline.decode(res['routes'][0]['geometry'])
        dprt_time =  end_time
        start_time = dprt_time.strftime('%Y-%m-%d-%H:%M:%S')
        #duration_time =  pd.to_datetime(df['dprt_time'][i])
        #end_time = pd.to_datetime(dprt_time + timedelta(seconds=res['routes'][0]['legs'][0]['duration'])).strftime('%Y-%m-%d-%H:%M:%S')
        route_gpd = gpd.GeoDataFrame(res['routes'][0]['legs'][0]['steps'])
        #duration_time =  pd.to_datetime(df['dprt_time'][i]) + timedelta(hours=9)
        drive = 1
        for j in range(route_gpd.shape[0]-1):
            detail_route = polyline.decode(route_gpd['geometry'][j])
            detail_duration = route_gpd['duration'][j] /len(detail_route)
            for k in range(len(detail_route)):
                lon = detail_route[k][1]
                lat = detail_route[k][0]
                duration_time = dprt_time + timedelta(seconds = detail_duration)
                dprt_time = duration_time
                new_df = new_df.append(pd.DataFrame([[car_num,duration_time,lon,lat,time_df['trip'][time_df.index[i]]+1,drive]], columns=['carnum','TIME','LNG','LAT','trip','driving']), ignore_index=True)
        new_df.to_csv('C:/Users/user/Desktop/Assignment/new_routetaxi.csv',encoding='utf-8')


  
            

In [13]:
a = new_df.loc[new_df['geometry'].isnull()]
a.to_csv('C:/Users/user/Desktop/Assignment/a333.csv',encoding='utf-8')

In [3]:
import pandas as pd
new_route = pd.read_csv('C:/Users/user/Desktop/Assignment/new_routetaxi.csv',encoding='utf-8')

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
new_route['driving'].value_counts()

1    310386
3    201661
2    195707
0     87118
Name: driving, dtype: int64

In [23]:
new_route.loc[new_route['carnum']=='탑승객']['driving'].value_counts()

3    201661
Name: driving, dtype: int64

In [4]:
new_route.head()

,Unnamed: 0,carnum,TIME,LAT,LNG,driving,trip,geometry
0,0,대전50바1226,2017-03-02 18:53:35,36.33938,127.39194,0,8,POINT (127.39194 36.33938)
1,1,대전50바1226,2017-03-02 18:54:15,36.33389,127.39862,0,8,POINT (127.39862 36.33389)
2,2,대전50바1226,2017-03-02 18:54:56,36.32582,127.39480,0,8,POINT (127.3948 36.32582)
3,3,대전50바1226,2017-03-02 18:55:37,36.32615,127.39639,0,8,POINT (127.39639 36.32615)
4,4,대전50바3735,2017-03-02 08:36:02,36.33171,127.39774,0,2,POINT (127.39774 36.33171)


In [6]:
len(new_route.loc[new_route['driving']==1]['carnum'].unique())

1549

In [8]:
new_route = new_df.copy()

In [9]:
#items는 배열, pivot은 입력한 시간 값
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

#해당 함수를 활용하기 위해서는 배열에 존재하는 값들의 형식과 pivot의 형식이 같아야 함
new_route['new_time'] = new_route['TIME'].apply(lambda x:pd.to_datetime(pd.to_datetime(x).strftime('%Y-%m-%d-%H:%M:%S')))

#pd.to_datetime을 두번사용하는 것은 협정 세계시간(UTC)를 없애기 위함이며 변하는 형식은 아래와 같음

In [10]:
df_near = pd.DataFrame()
df_near['carnum'] = None
df_near['TIME'] = None
df_near['LON'] = None
df_near['LAT'] = None
df_near['driving'] = None

In [11]:
seconds = [00,30]
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
for hour in tqdm(range(0,24)):
    for minute in range(0,60):
        for second in seconds:
            specific_time = pd.to_datetime(datetime(2017,3,2,hour,minute,second))
            specific_df = new_route.loc[(new_route['new_time']<specific_time+timedelta(seconds=15))&(new_route['new_time']>specific_time-timedelta(seconds=15))]
            specific_df = specific_df.drop_duplicates(['carnum'])
            specific_df['TIME'] = specific_time
            df_near = df_near.append(pd.DataFrame(specific_df))


100%|██████████| 24/24 [00:43<00:00,  1.80s/it]


In [12]:
df_near = df_near.sort_values(['carnum','TIME'])

In [13]:
df_near['driving'] = df_near['driving'].apply(lambda x:int(str(x).replace('0','1')))

In [ ]:
df_near['driving'].value_counts()

1    120501
2     25566
Name: driving, dtype: int64

In [14]:
dprt = pd.to_datetime(datetime(2017,3,2,6,0,0))
arri = pd.to_datetime(datetime(2017,3,2,15,0,0))
df_6to12 = df_near.loc[(df_near['new_time']>dprt)&(df_near['new_time']<arri)]
df_6to12.to_csv('C:/Users/user/Desktop/Assignment/df_6to12.csv',encoding='utf-8')

In [15]:
df_near.to_csv('C:/Users/user/Desktop/Assignment/df_near.csv',encoding='utf-8')

In [16]:
df_near = df_near.drop(['LON','Unnamed: 0','new_time'],axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [17]:
df_near_copy = df_6to12.copy()

In [18]:
#같은 시간대에 여러개의 위치 생성
from random import *
from tqdm import tqdm
import pandas as pd

for hour in tqdm(range(6,15)):
    for i in range(30):
        minute = randint(0,59)
        time = pd.Timestamp(2017,3,2,hour,minute,second)
        lat = uniform(36.28872150215423,36.411218105067874)
        lng = uniform(127.30846055394692,127.47428543061531)
        for j in range(10):
            carnum = None
            trip= None
            drive = 0
            empty_time = time + timedelta(seconds = (30*j))  
            df_near_copy = df_near_copy.append(pd.DataFrame([[empty_time,lat,lng,drive,carnum,trip]],
                                                            columns=['TIME','LAT','LNG','driving','carnum','trip']),ignore_index=True)
            #new_df = new_df.append(pd.DataFrame([[car_num,time_dif,lon,lat,drive]], columns=['carnum','TIME','LNG','LAT','driving']), ignore_index=True)

100%|██████████| 9/9 [00:21<00:00,  2.44s/it]


In [19]:
df_near

,carnum,TIME,LON,LAT,driving,LNG,trip,geometry,new_time
87036,대전50바1004,2017-03-02 00:40:00,NaN,36.324810,1,127.400560,1,POINT (127.40056 36.32481),2017-03-02 00:40:04
87037,대전50바1004,2017-03-02 00:40:30,NaN,36.327040,1,127.399680,1,POINT (127.39968 36.32704),2017-03-02 00:40:36
87038,대전50바1004,2017-03-02 00:41:00,NaN,36.329650,1,127.408470,1,POINT (127.40847 36.32965),2017-03-02 00:41:08
87039,대전50바1004,2017-03-02 00:41:30,NaN,36.334580,1,127.413390,1,POINT (127.41339 36.33458),2017-03-02 00:41:40
87040,대전50바1004,2017-03-02 00:42:00,NaN,36.342380,1,127.423310,1,POINT (127.42331 36.34238),2017-03-02 00:42:12
...,...,...,...,...,...,...,...,...,...
778207,탑승객,2017-03-02 21:05:30,NaN,36.393863,3,127.468705,5,NaN,2017-03-02 21:05:18
778223,탑승객,2017-03-02 21:06:00,NaN,36.393863,3,127.468705,5,NaN,2017-03-02 21:05:48
778239,탑승객,2017-03-02 21:06:30,NaN,36.393863,3,127.468705,5,NaN,2017-03-02 21:06:18
778255,탑승객,2017-03-02 21:07:00,NaN,36.393863,3,127.468705,5,NaN,2017-03-02 21:06:47


In [20]:
df_near_copy.to_csv('C:/Users/user/Desktop/Assignment/plus_empty_taxi.csv',encoding='utf-8')